In [ ]:
# Import the libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans, k_means
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
from plotly import graph_objects
from plotly import express
from plotly import figure_factory

In [ ]:
podo_data = pd.read_excel("C4C3-Podo-NewHeader.xlsx", header = [0, 1, 2])
podo_data.head()

In [ ]:
podo_df = podo_data.copy()
print("Podo dataset:", podo_df.shape)

In [ ]:
for col in podo_df.columns:
    print(col[0], "/-/", col[1], "/-/", col[2], " - & Number of NaN:", podo_df[col[0]][col[1]].isnull().sum()[0], "- & dtype:", podo_df[col].dtypes)
    print("----------------")

In [ ]:
podo_df = podo_df[podo_df["Accession"]["Protein names"]["Total_Inf_Accession_Protein.names"].notna()]
podo_df = podo_df[podo_df["Phospho ID Metrics"]["Phospho Sites"]["Phos_Inf_Phospho.ID.Metrics_Phosphosites"].notna()]
podo_df = podo_df[podo_df["No Header"]["Phosphopeptide Present"]["Both_Phosphopeptide.Present"] == True]
podo_df = podo_df[podo_df["Database"]["Human"]["Total_Database_Human"] == True]
podo_df.shape

In [ ]:
for col in podo_df.columns:
    print(col[0], "/-/", col[1], "/-/", col[2], " - & Number of NaN:", podo_df[col[0]][col[1]].isnull().sum()[0], "- & dtype:", podo_df[col].dtypes)
    print("----------------")